## Creation of the variables used in the classification

In [1]:
import pandas as pd

In [3]:
data_pan15_training = pd.read_csv(r'data\raw_data\PAN_15_training.csv')

In [19]:
data_pan15_test = pd.read_csv(r'data\raw_data\PAN_15_test.csv')

In [20]:
data_pan15_test

,Unnamed: 0,author,text,gender,age
0,0,02c49444-c779-40aa-8ef6-45455680c150,Was at ITsAP organized workshop for CEOs - won...,M,35-49
1,1,02c49444-c779-40aa-8ef6-45455680c150,"Intel Invests $30 Million in Cloud, Embedded R...",M,35-49
2,2,02c49444-c779-40aa-8ef6-45455680c150,"A management principle ""What is rewarded is re...",M,35-49
3,3,02c49444-c779-40aa-8ef6-45455680c150,SETU's new product myDrona - a personalized le...,M,35-49
4,4,02c49444-c779-40aa-8ef6-45455680c150,Which is a better place for conference banquet...,M,35-49
...,...,...,...,...,...
13173,13173,fc13c5fe-9028-4625-93a4-3e5c32098d89,@username So COOL!! I like you two together an...,F,50-XX
13174,13174,fc13c5fe-9028-4625-93a4-3e5c32098d89,@username @username @username @username @usern...,F,50-XX
13175,13175,fc13c5fe-9028-4625-93a4-3e5c32098d89,@username @username We have a saying here: if ...,F,50-XX
13176,13176,fc13c5fe-9028-4625-93a4-3e5c32098d89,Had a wish just now and it got fulfilled! YAY!...,F,50-XX


## Creating feature functions

In [5]:
import re
from nltk.probability import FreqDist
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import statistics


# Character-based features
def character_count(text):
    return len(text)

def alphabetic_ratio(text):
    alphabetic = sum(c.isalpha() for c in text)
    return alphabetic/len(text)

def uppercase_ratio(text):
    upper = sum(c.isupper() for c in text)
    return upper/len(text)

def digit_ratio(text):
    digit = sum(c.isdigit() for c in text)
    return digit/len(text)

def whitespace_ratio(text):
    whitespace = sum(c.isspace() for c in text)
    return whitespace/len(text)

def tab_ratio(text):
    tabs = text.count('\t')
    return tabs/len(text)

def letter_ratio(text, letter):
    text = text.lower()
    letter_count = text.count(letter)
    return letter_count/len(text)

def specialcharacter_ratio(text, character):
    spec_count = text.count(character)
    return spec_count/len(text)

# Word-based features
def number_words(text):
    words = re.findall(r'\b\w+\b', text)
    return len(words)

def word_length(text):
    words = re.findall(r'\b\w+\b', text)
    num_words = len(words)
    if num_words == 0:
        return 0

    total_length = sum(len(word) for word in words)
    return total_length/num_words

def vocabulary_richness(text):
    words = re.findall(r'\b\w+\b', text)
    num_words = len(words)
    if num_words == 0:
        return 0
    
    num_uniq_words = len(set(words))
    return num_uniq_words/num_words

def long_words(text):
    words = re.findall(r'\b\w+\b', text)
    long_words_list = [word for word in words if len(word) > 6]
    return len(long_words_list)/len(words)

def short_words(text):
    words = re.findall(r'\b\w+\b', text)
    short_words_list = [word for word in words if 1 <= len(word) <= 3]
    return len(short_words_list)/len(words)

def legomena(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    legomena = [word for word in freq if freq[word] == 1]
    return len(legomena)/len(words)

def dislegomena(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    dislegomena = [word for word in freq if freq[word] == 2]
    return len(dislegomena)/len(words)

def yules_k(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    N = len(words)
    Vi = FreqDist(freq.values())
    K = 10**4 * ((-N + sum(i**2 * Vi[i] for i in Vi))/N**2)
    return K

def simpson_d(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    N = len(words)
    if N < 2:
        return 0
    D = sum(fr * (fr - 1) / (N * (N - 1)) for fr in freq.values())
    return D

def sichel_s(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    dislegomena = [word for word in freq if freq[word] == 2]
    S = len(dislegomena)/len(freq.values())
    return S

def honores_r(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    N = len(words)
    V = len(freq.values())
    legomena = [word for word in freq if freq[word] == 1]
    R = (100*np.log(N)/(1-(len(legomena)/V)))
    return R

def entropy(text):
    words = re.findall(r'\b\w+\b', text.lower())
    freq = FreqDist(words)
    N = len(words)
    E = -sum((fr / N) * np.log(fr/N) for fr in freq.values())
    return E

# Syntatic features
def punctuations_ratio(text, punctuation):
    punctuation_list = re.findall(punctuation, text)
    return len(punctuation_list)/len(text)

# Structural features
def lines(text):
    return len(text.split('\n'))

def sentences(text):
    return len(sent_tokenize(text))

def paragraphs(text):
    return len([par for par in text.split('\n\n') if par.strip()])

def sentence_paragraph(text):
    pars = [par for par in text.split('\n\n') if par.strip()]
    return statistics.mean([len(sent_tokenize(par)) for par in pars])

def words_paragraph(text):
    pars = [par for par in text.split('\n\n') if par.strip()]
    return statistics.mean([len(re.findall(r'\b\w+\b', par)) for par in pars])

def chars_paragraph(text):
    pars = [par for par in text.split('\n\n') if par.strip()]
    return statistics.mean([len(par) for par in pars])

def words_sentences(text):
    sents = sent_tokenize(text)
    return statistics.mean([len(word_tokenize(sentence)) for sentence in sents])

def uppercase_start(text):
    sents = sent_tokenize(text)
    return (sum(1 for sentence in sents if sentence[0].isupper()) / len(sents))


In [10]:
def extract_features(dataframe, text_column):
    features = pd.DataFrame()

    # Character-based features
    features['total_characters'] = dataframe[text_column].apply(character_count)
    features['ratio_alphabetic'] = dataframe[text_column].apply(alphabetic_ratio)
    features['ratio_uppercase'] = dataframe[text_column].apply(uppercase_ratio)
    features['ratio_digit'] = dataframe[text_column].apply(digit_ratio)
    features['ratio_whitespace'] = dataframe[text_column].apply(whitespace_ratio)
    features['ratio_tabspace'] = dataframe[text_column].apply(tab_ratio)
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    for letter in letters:
        features[letter+'_frequency'] = dataframe[text_column].apply(letter_ratio, args=(letter,))
    special_characters = ['~', '@', '#', '$', '%', '^', '&', '*', '-', '_', '=', '+', '>', '<', '[', ']', '{', '}', '/', '\\', '|']
    for character in special_characters:
        features[character+'_frequency'] = dataframe[text_column].apply(specialcharacter_ratio, args=(character,))
    
    # Word-based features
    features['total_words'] = dataframe[text_column].apply(number_words)
    features['word_length'] = dataframe[text_column].apply(word_length)
    features['vocabulary_richness'] = dataframe[text_column].apply(vocabulary_richness)
    features['long_words'] = dataframe[text_column].apply(long_words)
    features['short_words'] = dataframe[text_column].apply(short_words)
    features['hapax_legomena'] = dataframe[text_column].apply(legomena)
    features['hapax_dislegomena'] = dataframe[text_column].apply(dislegomena)
    features['yules_k'] = dataframe[text_column].apply(yules_k)
    features['simpson_d'] = dataframe[text_column].apply(simpson_d)
    features['sichel_s'] = dataframe[text_column].apply(sichel_s)
    features['honore_r'] = dataframe[text_column].apply(honores_r)
    features['entropy'] =  dataframe[text_column].apply(entropy)
    # Brunet W?
    # word length frequency distribution

    
    # Syntactic features
    punctuations = [r"’", r",", r"\.", r":", r";", r"\?", r"\?{2,}", r"!", r"!{2,}", r"\.{3}"]
    for punctuation in punctuations:
        features[punctuation+"_frequency"] = dataframe[text_column].apply(punctuations_ratio, args=(punctuation,))

    # Structural features
    features['number_lines'] = dataframe[text_column].apply(lines)
    features['number_sentences'] = dataframe[text_column].apply(sentences)
    features['number_paragraphs'] = dataframe[text_column].apply(paragraphs)
    features['sentences_per_paragraph'] = dataframe[text_column].apply(sentence_paragraph)
    features['word_per_paragraph'] = dataframe[text_column].apply(words_paragraph)
    features['character_per_paragraph'] = dataframe[text_column].apply(chars_paragraph)
    features['word_per_sentence'] = dataframe[text_column].apply(words_sentences)
    features['ratio_sentencestart_uppercase'] = dataframe[text_column].apply(uppercase_start)
    features['gender'] = dataframe['gender']
    features['age'] = dataframe['age']
    
    return features

In [22]:
pan15_train = extract_features(data_pan15_training, 'text')

In [23]:
pan15_test = extract_features(data_pan15_test, 'text')

In [24]:
pan15_test.describe()

c:\Users\Sten\AppData\Local\R-MINI~1\envs\python39\lib\site-packages\numpy\lib\function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,total_characters,ratio_alphabetic,ratio_uppercase,ratio_digit,ratio_whitespace,ratio_tabspace,a_frequency,b_frequency,c_frequency,d_frequency,...,"!{2,}_frequency",\.{3}_frequency,number_lines,number_sentences,number_paragraphs,sentences_per_paragraph,word_per_paragraph,character_per_paragraph,word_per_sentence,ratio_sentencestart_uppercase
count,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,...,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000,13178.000000
mean,79.245409,0.728750,0.073417,0.014592,0.169534,0.034626,0.057177,0.012065,0.023249,0.020996,...,0.000569,0.001013,1.062225,1.647063,1.007133,1.640967,13.262603,78.897936,11.638788,0.548083
std,37.136378,0.060088,0.090831,0.023656,0.038658,0.025489,0.029597,0.014777,0.019565,0.019478,...,0.003905,0.004906,0.358864,1.002671,0.097526,0.992888,6.538972,37.094613,6.480261,0.436502
min,5.000000,0.210526,0.000000,0.000000,0.020270,0.010309,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,1.000000,0.000000
25%,48.000000,0.702703,0.024691,0.000000,0.141509,0.018018,0.037879,0.000000,0.008197,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,8.000000,47.000000,7.000000,0.000000
50%,76.000000,0.739130,0.052632,0.000000,0.164948,0.026316,0.054795,0.008475,0.021505,0.018182,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,13.000000,76.000000,10.000000,0.500000
75%,111.000000,0.768116,0.097222,0.022222,0.195122,0.041667,0.074074,0.019231,0.034884,0.031250,...,0.000000,0.000000,1.000000,2.000000,1.000000,2.000000,18.000000,111.000000,15.000000,1.000000
max,194.000000,0.948052,0.903226,0.317073,0.592593,0.400000,0.441176,0.200000,0.181818,0.372549,...,0.090909,0.117647,8.000000,11.000000,4.000000,11.000000,34.000000,194.000000,44.333333,1.000000


In [25]:
pan15_test.to_csv(r'data\pan15_features_test.csv', index=False)